In [ ]:
include("constructor.jl")

In [ ]:
struct Function
    name
    outputs
    inputs
    lines
end

struct Line
    operation
    condition
    outputs
    inputs
    conditionals
end

struct Var
    id
    lifetime
    scope
end

struct Const
    value
end

struct Flabel
    id
end

In [ ]:
function freePos(mempos, memzero = 1)
    pos = memzero
    vals = collect(values(mempos))
    while true
        if (pos in vals) == false
            return pos
        end
        pos = pos+1
    end
end

In [ ]:
function assignVarmap(f)
    varmap = Dict()
    for (line, k) in f.lines[1:end-1] |> x -> zip(x, 0:length(x))
        if line.outputs != nothing
            for var in line.outputs
                if typeof(var) != Const && haskey(varmap, var) == false
                    push!(varmap, var => Var(var, k, f.name))
                end
            end
        end
    end
    return varmap
end

In [ ]:
function choose(f, mempos, functions=Dict())
    preasm = []
    varmap = assignVarmap(f)
    for (line, k) in reverse(f.lines) |> x -> zip(x, reverse(0:length(x)-1))
        if line.operation == 1
            for (var, output) in zip(line.inputs, f.outputs)
                push!(mempos, varmap[var] => mempos[output])
                delete!(mempos, output)
            end
        elseif line.operation == 0
            for (var, input) in zip(line.outputs, f.inputs)
                push!(mempos, input => mempos[varmap[var]])
                delete!(mempos, varmap[var])
            end 
        elseif typeof(line.operation) == Flabel
            if line.operation.id != f.name
                push!(preasm, Line(2, "}", nothing, nothing, nothing))
                
                templatefn = functions[line.operation.id]
                inputvars = line.inputs == nothing ? nothing : [varmap[var] for var in line.inputs]
                outputvars = line.outputs == nothing ? nothing : [varmap[var] for var in line.outputs]
                tempfn = Function(templatefn.name, outputvars, inputvars, templatefn.lines)
                for subline in reverse(choose(tempfn, mempos, functions))
                    push!(preasm, subline)
                end
                
                push!(preasm, Line(2, "{", nothing, nothing, nothing))
                if line.condition != nothing
                    
                    condvars = line.conditionals == nothing ? nothing : line.conditionals
                    for var in condvars
                        if typeof(var) != Const
                            if haskey(mempos, varmap[var]) == false
                                push!(mempos, varmap[var] => freePos(mempos))
                            end
                        end
                    end
                    condpos = line.conditionals == nothing ? nothing : [typeof(var) == Const ? var : mempos[varmap[var]] for var in line.conditionals]
                    push!(preasm, Line(3, line.condition, nothing, nothing, condpos))
                end
                
            else #haram turbo mega logikbombe
                inputvars = line.inputs == nothing ? [] : line.inputs
                inputvars = line.conditionals == nothing ? inputvars : [inputvars; line.conditionals]
                for var in inputvars
                    if typeof(var) != Const
                        if haskey(mempos, varmap[var]) == false
                            push!(mempos, varmap[var] => freePos(mempos))
                        end
                    end
                end
                
                condpos = line.conditionals == nothing ? nothing : [typeof(var) == Const ? var : mempos[varmap[var]] for var in line.conditionals]
                push!(preasm, Line(4, line.condition, nothing, nothing, condpos))
            end #haram turbo mega logikbombe ende
            
        elseif typeof(line.operation) == String
            inputvars = line.inputs == nothing ? [] : line.inputs
            inputvars = line.conditionals == nothing ? inputvars : [inputvars; line.conditionals]
            outputvars = line.outputs == nothing ? [] : deepcopy(line.outputs)
            for var in inputvars
                if typeof(var) != Const
                    if haskey(mempos, varmap[var]) == false
                        
                        to_replace = true
                        for (outvar, s) in zip(outputvars, 1:length(outputvars))
                            if typeof(outvar) != Const && varmap[outvar].lifetime >= k
                                push!(mempos, varmap[var] => mempos[varmap[outvar]])
                                deleteat!(outputvars, s)
                                to_replace = false
                                break
                            end
                        end
                        
                        if to_replace == true
                            push!(mempos, varmap[var] => freePos(mempos))
                        end
                        
                    end
                end
            end
            
            outpos = line.outputs == nothing ? nothing : [typeof(var) == Const ? var : mempos[varmap[var]] for var in line.outputs]
            inpos = line.inputs == nothing ? nothing : [typeof(var) == Const ? var : mempos[varmap[var]] for var in line.inputs]
            condpos = line.conditionals == nothing ? nothing : [typeof(var) == Const ? var : mempos[varmap[var]] for var in line.conditionals]
            push!(preasm, Line(line.operation, line.condition, outpos, inpos, condpos))
            
            outputvars = line.outputs == nothing ? [] : line.outputs
            for var in outputvars
                if typeof(var) != Const && varmap[var].lifetime >= k #off by 1 farming
                     delete!(mempos, varmap[var])
                end
            end
        else
            println("invalid operation")
        end
    end
    return reverse(preasm)
end

In [ ]:
mempos = Dict()

lh = [
    Line(0, nothing, [1, 2, 3], nothing, nothing),
    Line("", "=", [4], [1], [3]),
    Line("dec", nothing, [3], [3], nothing),
    Line("", "=", [4], [2], [3]),
    Line(1, nothing, nothing, [1, 2, 3, 4], nothing)
]
h = Function("h", nothing, nothing, lh)
display(h.lines)

lg = [
    Line(0, nothing, [1, 2, 3], nothing, nothing),
    Line("+", nothing, [1], [1, 2], nothing),
    Line("", nothing, [4], [2], nothing),
    Line("", nothing, [2], [1], nothing),
    Line("", nothing, [1], [4], nothing),
    Line("dec", nothing, [3], [3], nothing),
    Line(Flabel("g"), ">=", nothing, [1, 2, 3], [3]),
    Line(1, nothing, nothing, [1], nothing)
]
g = Function("g", nothing, nothing, lg)
display(g.lines)

lmain = [
    Line(0, nothing, [], nothing, nothing),
    Line("", nothing, [1], [Const("read")], nothing),
    Line("", nothing, [2], [Const("read")], nothing),
    Line("", nothing, [3], [Const("read")], nothing),
    Line(Flabel("h"), nothing, [4, 5, 6, 7], [1, 2, 3], nothing),
    Line("", nothing, ["tempval"], [7], nothing),
    Line("", nothing, ["temp"], [6], nothing),
    Line(Flabel("g"), ">", [8], [4, 5, 6], [6]),
    Line("", "<=", [8], ["tempval"], ["temp"]),
    Line("write", nothing, nothing, [8], nothing),
    Line(1, nothing, nothing, [], nothing)
]
main = Function("main", [], [], lmain)
display(main.lines)

functions = Dict()
push!(functions, "g" => g)
push!(functions, "h" => h)

In [ ]:
mempos = Dict()

lh = [
    Line(0, nothing, [1, 2, 3], nothing, nothing),
    Line("", "=", [1], [1], [3]),
    Line("dec", nothing, [3], [3], nothing),
    Line("", "=", [1], [2], [3]),
    Line(1, nothing, nothing, [1, 2, 3], nothing)
]
h = Function("h", nothing, nothing, lh)
display(h.lines)

lg = [
    Line(0, nothing, [1, 2, 3], nothing, nothing),
    Line("+", nothing, [1], [1, 2], nothing),
    Line("", nothing, [4], [2], nothing),
    Line("", nothing, [2], [1], nothing),
    Line("", nothing, [1], [4], nothing),
    Line("dec", nothing, [3], [3], nothing),
    Line(Flabel("g"), ">=", nothing, [1, 2, 3], [3]),
    Line(1, nothing, nothing, [1, 2, 3], nothing)
]
g = Function("g", nothing, nothing, lg)
display(g.lines)

lmain = [
    Line(0, nothing, [], nothing, nothing),
    Line("", nothing, [1], [Const("read")], nothing),
    Line("", nothing, [2], [Const("read")], nothing),
    Line("", nothing, [3], [Const("read")], nothing),
    Line(Flabel("h"), nothing, [1, 2, 3], [1, 2, 3], nothing),
    Line(Flabel("g"), ">", [1, 2, 3], [1, 2, 3], [3]),
    Line("write", nothing, nothing, [1], nothing),
    Line("write", nothing, nothing, [2], nothing),
    Line("write", nothing, nothing, [3], nothing),
    Line(1, nothing, nothing, [], nothing)
]
main = Function("main", [], [], lmain)
display(main.lines)

functions = Dict()
push!(functions, "g" => g)
push!(functions, "h" => h)

In [ ]:
mempos = Dict()

lh = [
    Line(0, nothing, [1, 2], nothing, nothing),
    Line("inc", nothing, [1], [1], nothing),
    Line("dec", nothing, [2], [2], nothing),
    Line(1, nothing, nothing, [1, 2], nothing)
]
h = Function("h", nothing, nothing, lh)
display(h.lines)

lg = [
    Line(0, nothing, [1, 2, 3], nothing, nothing),
    Line(Flabel("h"), nothing, [1, 2], [1, 2], nothing),
    Line("dec", nothing, [3], [3], nothing),
    Line(Flabel("g"), ">", nothing, [1, 2, 3], [3]),
    Line(1, nothing, nothing, [1, 2, 3], nothing)
]
g = Function("g", nothing, nothing, lg)
display(g.lines)

lmain = [
    Line(0, nothing, [], nothing, nothing),
    Line("", nothing, [1], [Const("read")], nothing),
    Line("", nothing, [2], [Const("read")], nothing),
    Line("", nothing, [3], [Const("read")], nothing),
    Line(Flabel("g"), ">", [1, 2], [1, 2, 3], [3]),
    Line("write", nothing, nothing, [1], nothing),
    Line("write", nothing, nothing, [2], nothing),
    Line(1, nothing, nothing, [], nothing)
]
main = Function("main", [], [], lmain)
display(main.lines)

functions = Dict()
push!(functions, "g" => g)
push!(functions, "h" => h)

In [ ]:
mempos = Dict()

lf = [
    Line(0, nothing, [1, 2], nothing, nothing),
    Line("+", nothing, [1], [1, Const(2)], nothing),
    Line("write", nothing, nothing, [1], nothing),
    Line("write", nothing, nothing, [2], nothing),
    Line("", nothing, [3], [Const(0)], nothing),
    Line(Flabel("f"), "=", nothing, [1, 2], [3]),
    Line(1, nothing, nothing, [], nothing)
]
f = Function("f", nothing, nothing, lf)
display(f.lines)

lmain = [
    Line(0, nothing, [], nothing, nothing),
    Line("", nothing, [1], [Const("read")], nothing),
    Line("", nothing, [2], [Const("read")], nothing),
    Line(Flabel("f"), nothing, nothing, [1, 2], nothing),
    Line(1, nothing, nothing, [], nothing)
]
main = Function("main", [], [], lmain)
display(main.lines)

functions = Dict()
push!(functions, "f" => f)

In [ ]:
preasm = choose(main, mempos, functions)

In [ ]:
mempos

In [ ]:
create(preasm, "w")